# Use DeepCCA to transform ADNI features

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import torch
import torch.nn as nn
import numpy as np
from linear_cca import linear_cca
from torch.utils.data import BatchSampler, SequentialSampler
from DeepCCAModels import DeepCCA
from main import Solver
from utils import load_data, svm_classify
try:
    import cPickle as thepickle
except ImportError:
    import _pickle as thepickle
from IPython.display import display

### Read the database, examine it:

Instead of reading the whole database, we read only the data that's useful to us. That is, we read only specific columns of data, and we take only the row containing the first scan for each person. 

In "ADNI Regressional Analysis.ipynb" we have done that exactly, as well as performed linear regression transformation to the imaging data, in order to remove any age, sex, and DLICV_baseline effect. 

Furthermore, in "ADNI OPNMF.ipynb" we have performed dimensionality reduction through the OPNMF method, reducing the number of the ROIs from 145 to just 18. 

The data is located at "./DATA/Reduced_Linearly_Transformed_Unique_Dataset.pkl"

In [2]:
unique = pd.read_pickle("./DATA/Linearly_Transformed_Unique_Dataset.pkl")
unique.head(15)

,PTID,Date,Age,Sex,DLICV_baseline,APOE4_Alleles,APOE_Genotype,Diagnosis_nearest_2.0,MUSE_Volume_4,MUSE_Volume_11,...,rs28394864,rs111278892,rs3752246,rs4147929,rs41289512,rs3865444,rs6024870,rs6014724,rs7274581,rs429358
0,002_S_0295,2006-04-18,84.742466,0,1485405.375,1.0,E3/E4,CN,-440.777069,-507.297168,...,0,1,1,1,0,0,0,0,0,1
9,002_S_0413,2006-05-02,76.283562,1,1364116.000,0.0,E3/E3,CN,577.755137,-188.813792,...,1,0,1,1,0,1,0,0,0,0
24,002_S_0559,2006-05-23,79.223288,0,1570479.625,1.0,E3/E4,CN,198.499249,1080.290951,...,2,0,0,0,0,1,0,0,0,0
31,002_S_0619,2006-06-01,77.447945,0,1859348.250,2.0,E4/E4,Dementia,2623.687012,649.558822,...,1,0,0,0,1,1,0,0,0,2
36,002_S_0685,2006-07-06,89.561644,1,1372862.125,0.0,E3/E3,CN,727.400228,-250.782773,...,2,1,1,1,0,0,0,0,0,0
45,002_S_0729,2006-07-17,65.056164,1,1166961.750,1.0,E3/E4,MCI,246.226215,628.340793,...,1,0,0,0,1,1,0,0,0,1
64,002_S_0816,2006-08-30,70.767123,0,1444128.125,2.0,E4/E4,Dementia,-145.138564,-193.593195,...,0,0,0,0,0,1,0,0,0,2
69,002_S_0938,2006-10-05,82.167123,1,1309685.000,0.0,E3/E3,Dementia,169.421397,-610.085153,...,0,0,1,1,0,1,0,0,0,0
74,002_S_0954,2006-10-10,69.198630,1,1075661.500,1.0,E3/E4,MCI,-81.664210,1343.833768,...,2,2,1,1,0,1,0,0,0,1
81,002_S_0955,2006-10-11,78.161644,1,1363607.000,1.0,E3/E4,Dementia,1036.385233,-353.324662,...,1,1,0,0,0,1,0,0,0,1


### Create the 2 views:

The first view consists of the imaging data, that are in the form of 145 real numbers. Those numbers are based on a prediction from a Linear Regression estimator trained only on the Cognitive Normal datapoints. The predictions then are subtracted from the actual values, and the remaining value (residual) is the datapoint for each ROI.

The second view consists of the 54 SNP (Single Nucleotide Polymorphism, "snip"), for each individual. They are either 0 or 1. 

The 2 views are the most basic views that can be used for the Deep CCA, and in further tests more features will be included.

In [18]:
# View 1:
view_1 = unique.loc[:,"MUSE_Volume_4":"MUSE_Volume_207"]

# View 2:
view_2 = unique.loc[:,"rs4575098":"rs429358"]

In [27]:
print("View 1:")
display(view_1.head())
print("View 2:")
display(view_2.head())

View 1:


,MUSE_Volume_4,MUSE_Volume_11,MUSE_Volume_23,MUSE_Volume_30,MUSE_Volume_31,MUSE_Volume_32,MUSE_Volume_35,MUSE_Volume_36,MUSE_Volume_37,MUSE_Volume_38,...,MUSE_Volume_198,MUSE_Volume_199,MUSE_Volume_200,MUSE_Volume_201,MUSE_Volume_202,MUSE_Volume_203,MUSE_Volume_204,MUSE_Volume_205,MUSE_Volume_206,MUSE_Volume_207
0,-440.777069,-507.297168,-35.171797,-14.510211,90.244138,140.075639,299.828133,63.889680,56.259492,4434.963481,...,745.557312,-188.954470,-1594.432454,-1648.308374,798.003198,-468.672456,-81.798945,283.990527,-134.708868,-102.291612
9,577.755137,-188.813792,35.574764,-39.881572,40.161648,58.255314,-909.956510,-107.325098,118.445748,-932.590538,...,1336.384182,2631.004114,1410.754665,30.295558,-1258.071206,115.187177,-175.177715,-533.517736,-37.990106,-475.586534
24,198.499249,1080.290951,137.416288,142.586830,121.231074,41.449232,1825.886437,-267.694901,6.605333,-947.176391,...,-1764.158370,-2206.292278,1473.087979,532.054466,1714.763199,2469.640085,209.533224,-49.858132,-206.268764,-117.520261
31,2623.687012,649.558822,-162.939446,-122.191780,-329.934406,-351.510297,-3426.992838,-826.297201,-713.213854,-355.750507,...,-641.454806,583.322773,-701.560285,-1369.412583,-2919.253412,-2766.270514,-757.912814,-822.771500,-347.672981,-131.863034
36,727.400228,-250.782773,-27.195126,6.992014,48.594085,57.675286,1120.707525,136.378936,316.063586,4364.772160,...,47.451381,279.144879,-1423.470776,-1844.920979,-186.886077,-270.949818,-312.293181,174.818988,-559.090352,-289.513744


View 2:


,rs4575098,rs6656401,rs2093760,rs4844610,rs4663105,rs6733839,rs10933431,rs35349669,rs6448453,rs190982,...,rs28394864,rs111278892,rs3752246,rs4147929,rs41289512,rs3865444,rs6024870,rs6014724,rs7274581,rs429358
0,0,0,0,0,1,1,1,0,0,1,...,0,1,1,1,0,0,0,0,0,1
9,1,0,0,0,0,0,0,1,0,0,...,1,0,1,1,0,1,0,0,0,0
24,0,0,0,0,1,0,1,0,0,1,...,2,0,0,0,0,1,0,0,0,0
31,0,1,1,1,0,0,0,2,1,1,...,1,0,0,0,1,1,0,0,0,2
36,0,0,0,0,0,0,1,1,0,0,...,2,1,1,1,0,0,0,0,0,0


### Parameters:

In [32]:
# if a gpu exists, torch.device should be 'gpu'
if torch.cuda.is_available():
    device = torch.device('gpu')
else:
    device = torch.device('cpu')
# print("Using", torch.cuda.device_count(), "GPUs")

# the path to save the final learned features
save_to = './DATA/ADNI_initial_features.gz'

# the size of the new space learned by the model (number of the new features)
outdim_size = 10

# size of the input for view 1 and view 2
input_shape1 = 145 # view_1.shape[1]
input_shape2 = 54  # view_2.shape[2]

# number of layers with nodes in each one
# this apparently can be different for each network, some experimentation is needed!
layer_sizes1 = [1024, 1024, outdim_size]
layer_sizes2 = [1024, 1024, outdim_size]

# the parameters for training the network
learning_rate = 1e-3
epoch_num = 100
batch_size = 2000

# the regularization parameter of the network
# seems necessary to avoid the gradient exploding especially when non-saturating activations are used
reg_par = 1e-5

# specifies if all the singular values should get used to calculate the correlation or just the top 
# outdim_size ones
# if one option does not work for a network or dataset, try the other one
use_all_singular_values = False

# if a linear CCA should get applied on the learned features extracted from the networks
# it does not affect the performance on noisy MNIST significantly
apply_linear_cca = True

###  Building, training, and producing the new features by DCCA

In [33]:
# Convert the pandas dataframe to numpy arrays for pytorch:
view_1_n = view_1.to_numpy()
view_2_n = view_2.to_numpy()

In [34]:
# Scramble the datapoints for randomness:
indices = np.arange(view_1_n.shape[0])
np.random.shuffle(indices)
view_1_n = view_1_n[indices]
view_2_n = view_2_n[indices]

print(view_1_n.shape, type(view_1_n), view_1_n.dtype)
print(view_2_n.shape, type(view_2_n), view_2_n.dtype)

view_1_t = torch.from_numpy(view_1_n)
print(view_1_t.shape, type(view_1_t))
view_2_t = torch.from_numpy(view_2_n)
print(view_2_t.shape, type(view_2_t))

(1569, 145) <class 'numpy.ndarray'> float64
(1569, 54) <class 'numpy.ndarray'> int64
torch.Size([1569, 145]) <class 'torch.Tensor'>
torch.Size([1569, 54]) <class 'torch.Tensor'>
